In [5]:
import os
import openai

client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))


vector_store = client.beta.vector_stores.create(name="축구 규칙 파일")

file_streams = open("축구규칙정리.pdf", "rb")

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(vector_store_id=vector_store.id, files=[file_streams])

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [6]:


instruction = '''
[목적]
이 GPT는 축구 규칙을 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한
질문만 부탁해요^^"라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요
예시)
- 질문: 질문 내용
- 답변: 답변 내용
4. 모든 질문에 한국어로 답변해주세요.
'''

assistant = client.beta.assistants.create(
    name='축구 규칙 설명 챗봇',
    instructions=instruction,
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

print(assistant.id)


asst_AKaZLtBcSYHmUcCc9kMVo3Qw


In [7]:
my_assistants = client.beta.assistants.list(order="desc", limit=20)
print(my_assistants.data)

[Assistant(id='asst_AKaZLtBcSYHmUcCc9kMVo3Qw', created_at=1732703858, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한\n질문만 부탁해요^^"라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n- 질문: 질문 내용\n- 답변: 답변 내용\n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-4o-mini', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_xFjafyj8uBGiV2wRvWMnoPAH'])), top_p=1.0), Assistant(id='asst_XXpfV4Mm7f8cZvyeBlT23PVH', created_at=1703051728, description=None, instructions='유저가 주는 단어나 문구가 영어가 아닌 경우 영어로 번역하고, 그 단어나 문구가 포함되는 영어 문장 10개와 각각의 문장에 대한 한국어 번역을 작성해줘.',

In [17]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_EpXC3rNmGKDCbFZI0FUNclPs', created_at=1732704844, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [18]:
message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content="축구장의 크기는?")

In [19]:
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [20]:
import time

while run.status not in ["completed", "failed"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(run.status)
    time.sleep(5)


in_progress
in_progress
completed


In [21]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [22]:
for each in messages:
    print(each.role + ": " + each.content[0].text.value)
    print("============")

assistant: - 질문: 축구장의 크기는?
- 답변: 축구장의 크기는 다음과 같습니다:
  - 터치라인 길이:
    - 최소 90m (100 야드)
    - 최대 120m (130 야드)
  - 골라인 길이:
    - 최소 45m (50 야드)
    - 최대 90m (100 야드)
  - 국제경기를 위한 크기:
    - 터치라인: 최소 100m (110 야드), 최대 110m (120 야드)
    - 골라인: 최소 64m (70 야드), 최대 75m (80 야드)【4:0†source】.
user: 축구장의 크기는?


In [3]:

client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content="오프사이드 룰에 대해 설명해줘")

messages = client.beta.threads.messages.list(thread_id=thread.id)

NameError: name 'openai' is not defined

In [27]:
for each in messages:
    print(each.role + ": " + each.content[0].text.value)
    print("============")

user: 오프사이드 룰에 대해 설명해줘


In [5]:
import openai
import os

client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
response = client.beta.assistants.delete("asst_AKaZLtBcSYHmUcCc9kMVo3Qw")

In [7]:
file_list = client.files.list()
print(file_list)

SyncCursorPage[FileObject](data=[FileObject(id='file-DGrVGGkMkwjB8BDWKXYVp9', bytes=22073717, created_at=1732703838, filename='축구규칙정리.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False, first_id='file-DGrVGGkMkwjB8BDWKXYVp9', last_id='file-DGrVGGkMkwjB8BDWKXYVp9')


In [11]:
file_id = file_list.data[0].id
print(file_id)

file-DGrVGGkMkwjB8BDWKXYVp9


In [9]:
response = client.files.delete(file_id)
print(response)

FileDeleted(id='file-DGrVGGkMkwjB8BDWKXYVp9', deleted=True, object='file')


In [16]:
import yfinance as yf

# 티커 객체 생성 (예: 애플 주식)
ticker = yf.Ticker("AAPL")

# 시세 데이터 가져오기
hist = ticker.history(period="5d")  # 최근 1년 데이터
print(hist.head())

# 재무 정보 가져오기
info = ticker.info
print(info['sector'])  # 섹터 정보 출력
print(info['dividendYield'])  # 배당 수익률 출력

                                 Open        High         Low       Close  \
Date                                                                        
2024-11-20 00:00:00-05:00  228.059998  229.929993  225.889999  229.000000   
2024-11-21 00:00:00-05:00  228.880005  230.160004  225.710007  228.520004   
2024-11-22 00:00:00-05:00  228.059998  230.720001  228.059998  229.869995   
2024-11-25 00:00:00-05:00  231.460007  233.250000  229.740005  232.869995   
2024-11-26 00:00:00-05:00  233.330002  235.570007  233.330002  235.059998   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-11-20 00:00:00-05:00  35169600        0.0           0.0  
2024-11-21 00:00:00-05:00  42108300        0.0           0.0  
2024-11-22 00:00:00-05:00  38168300        0.0           0.0  
2024-11-25 00:00:00-05:00  90152800        0.0           0.0  
2024-11-26 00:00:00-05:00  45897300        0.0           0.0  
Technology
0.0043
